In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import torch
from copy import deepcopy
import json, pickle as pkl
import numpy as np
import random
from dataset import COCODataset, COCODataLoader, ProcessedCOCOExample
from PIL import Image

In [ ]:
# 1. check dataloader
ds = COCODataset(split="train", max_samples=None, filterids={460872, 494550})
#ds2 = COCODataset(split="valid", max_samples=None, filterids={460872, 494550})

Found annotations at '/home/lukovdg1/fiftyone/coco-2017/raw/instances_train2017.json'
Images already downloaded
Existing download of split 'train' is sufficient
Loading existing dataset 'coco-2017-train-None'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use
Filtering ids: 2


 34%|█████████████████████████████████████████████████████████████████████████████▌                                                                                                                                                   | 31288/90816 [02:07<04:10, 237.67it/s]

In [8]:
print("qdf")

In [18]:
def crop_and_resize(x, size=512):
    if x.size == (size, size):
        return x
    w, h = x.size
    print(w, h)
    s = min(w, h)
    # compute maximum centercrop
    w_remain, h_remain = w - s, h - s
    x = x.crop((round(w_remain / 2), round(h_remain / 2), w - (w_remain - round(w_remain / 2)), h - (h_remain - round(h_remain / 2))))
    print(x.size)
    x = x.resize((size, size))
    print(x.size)
    return x
    

In [19]:
catdogexample = deepcopy(ds.examples[0][1][0])
catdogexample.image_data = crop_and_resize(catdogexample.load_image())
catdogexample.image_path = None
catdogexample.regions =[[np.array(crop_and_resize(Image.fromarray(region[0]))), region[1]] for region in catdogexample.regions]

catdogexample.captions = ["a white dog sitting next to a grey cat"]
print(catdogexample.captions)
# print(catdogexample.regions)
# print(catdogexample.regions[0])
print([x[1] for x in catdogexample.regions])
del catdogexample.regions[-1]

print([x[1] for x in catdogexample.regions])
print(catdogexample.regions[0][0].shape)
catdogcopy = deepcopy(catdogexample)
catdogcopy.regions = [[x for x in regionse] for regionse in catdogcopy.regions]
catdogcopy.regions[0][1] = catdogexample.regions[1][1]
catdogcopy.regions[1][1] = catdogexample.regions[0][1]
print(catdogexample.regions)
print(catdogcopy.regions)

In [20]:
display(catdogcopy.load_image())
display(Image.fromarray(catdogcopy.regions[0][0]))
print(catdogcopy.load_image().size)
# ds.examples[0][1][0].regions

In [21]:
applesorangesexample = deepcopy(ds.examples[0][1][1])
display(applesorangesexample.load_image())
print(applesorangesexample.load_image().size)

applesorangesexample.image_data = crop_and_resize(applesorangesexample.load_image())
applesorangesexample.image_path = None
applesorangesexample.regions =[[np.array(crop_and_resize(Image.fromarray(region[0]))), region[1]] for region in applesorangesexample.regions]

applesorangesexample.captions = ["a red orange sitting beside a green apple"]
print(applesorangesexample.captions)
# print(catdogexample.regions)
# print(catdogexample.regions[0])
print(applesorangesexample.regions[0][0].shape)

print([x[1] for x in applesorangesexample.regions])
print([x[0].astype("float").sum() for x in applesorangesexample.regions])
for region in applesorangesexample.regions:
    pass #display(Image.fromarray(region[0]))
    
del applesorangesexample.regions[-1]
del applesorangesexample.regions[0]
applesorangesexample.regions[1][1] = "apple"

applesorangescopy = deepcopy(applesorangesexample)
applesorangescopy.regions = [[x for x in regionse] for regionse in applesorangescopy.regions]
applesorangescopy.regions[0][1] = applesorangesexample.regions[1][1]
applesorangescopy.regions[1][1] = applesorangesexample.regions[0][1]
print(applesorangesexample.regions)
print(applesorangescopy.regions)

In [22]:
def display_example(x):
    img = x.load_image()
    print(img.size)
    display(img)
    for caption in x.captions:
        print(caption)
    for region in x.regions:
        print(f"Region {region[1]} of size {region[0].shape}")
        display(Image.fromarray(region[0]))

In [23]:
display_example(applesorangesexample)
display_example(applesorangescopy)

In [24]:
display_example(catdogexample)
display_example(catdogcopy)

In [25]:
with open("coco_dev_examples.dump", "wb") as f:
    pkl.dump([applesorangesexample, applesorangescopy, catdogexample, catdogcopy], f)

In [84]:
np.all(np.array(Image.fromarray(catdogcopy.regions[0][0])) == catdogcopy.regions[0][0])

True

In [85]:
round(0.5)

0

In [19]:
pkl.dumps(ds)

PicklingError: Can't pickle <class 'dataset.COCODataset'>: it's not the same object as dataset.COCODataset

In [ ]:
class MyCLIPTextEmbedder(FrozenCLIPEmbedder):
    """Uses the CLIP transformer encoder for text (from huggingface)"""

    def forward(self, fullprompt):
        fullprompt, batsize = fullprompt if isinstance(fullprompt, tuple) else (fullprompt, 1)
        
        layer_idses = []
        text_embeddingses = []
        token_idses = []
        
        layertexts = fullprompt.split("\n|")
        
        # encode layers separately
        for i, pos_prompt in enumerate(layertexts):
            if i == 0:
                token_ids, layer_ids = _tokenize_annotated_prompt(pos_prompt.strip(), self.tokenizer)
                global_len = token_ids.shape[0]
                global_bos_pos = 0
                global_eos_pos = torch.nonzero(token_ids == self.tokenizer.eos_token_id)[0][0]
            else:
                pos_prompt = pos_prompt.strip()
                token_ids = self.tokenizer([pos_prompt], return_tensors="pt",
                                        max_length=self.max_length, return_overflowing_tokens=False,
                                        truncation=True)["input_ids"][0]
                layer_ids = torch.tensor([i] * token_ids.shape[0])
            outputs = self.transformer(input_ids=token_ids[None].to(self.device), output_hidden_states=self.layer=="hidden")
            if self.layer == "last":
                z = outputs.last_hidden_state
            elif self.layer == "pooled":
                z = outputs.pooler_output[:, None, :]
            else:
                z = outputs.hidden_states[self.layer_idx]
                
            layer_idses.append(layer_ids)
            token_idses.append(token_ids)
            text_embeddingses.append(z)
            
        layer_ids = torch.cat(layer_idses, 0)[None].repeat(batsize, 1)
        token_ids = torch.cat(token_idses, 0)[None].repeat(batsize, 1)
        text_embeddings = torch.cat(text_embeddingses, 1).repeat(batsize, 1, 1)
        global_prompt_mask = torch.zeros_like(token_ids)
        global_bos_eos_mask = torch.zeros_like(global_prompt_mask)
        global_prompt_mask[:, :global_len] = 1
        global_bos_eos_mask[:, global_bos_pos] = 1
        global_bos_eos_mask[:, global_eos_pos] = 1
        
        ret = CustomTextConditioning(text_embeddings, layer_ids.to(self.device), token_ids.to(self.device),
                                     global_prompt_mask.to(self.device), global_bos_eos_mask.to(self.device))
        return ret
        # return text_embeddings, layer_ids.to(self.device), token_ids.to(self.device)


In [ ]:
# 2. create model
model_name = 'control_v11p_sd15_seg'
model = create_model(f'./models/{model_name}.yaml').cpu()
# load main weights
model.load_state_dict(load_state_dict('./models/v1-5-pruned.ckpt', location='cuda'), strict=False)
# load controlnet weights
model.load_state_dict(load_state_dict(f'./models/{model_name}.pth', location='cuda'), strict=False)
model = model.cuda()

# cast text encoder to our own
model.cond_stage_model.__class__ = MyCLIPTextEmbedder

for _module in model.model.diffusion_model.modules():
    if _module.__class__.__name__ == "CrossAttention":
        _module.__class__.forward = custom_forward
